In [1]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle
import time
import tqdm

import data_utils
import model_utils_train
import model_utils_infer
import train_utils
import evaluation
importlib.reload(data_utils)
importlib.reload(model_utils_train)
importlib.reload(model_utils_infer)


importlib.reload(train_utils)
importlib.reload(evaluation)

import torch
import torch.nn as nn

from transformers import DistilBertPreTrainedModel
from transformers import DistilBertForQuestionAnswering
from transformers import DistilBertConfig, DistilBertTokenizer

### Load data

In [2]:
file_dict = {"train1.1tiny": "./data/tiny_train-v1.1.json",
             "train2.0tiny": "./data/tiny_train-v2.0.json",
             "train1.1": "./data/train-v1.1.json", 
             "dev1.1": "./data/dev-v1.1.json", 
             "train1.1medium": "./data/medium_train_v1.1.json",
             "dev1.1medium": "./data/medium_dev_v1.1.json"}
train_data_v1_debug = data_utils.prep_data(file_dict["train1.1tiny"])
train_data_v2_debug = data_utils.prep_data(file_dict["train2.0tiny"])
train_data_v1 = data_utils.prep_data(file_dict["train1.1medium"])
dev_data_v1 = data_utils.prep_data(file_dict["dev1.1medium"])

### Run inference with Huggingface pretrained BERT fine-tuned on SQuAD

In [ ]:
importlib.reload(model_utils)
importlib.reload(train_utils)
m = model_utils.PreTrainedSQuAD()
result = train_utils.inference(train_data_v2, m)

### Run inference with Huggingface pretrained DistillBERT fine-tuned on SQuAD

In [ ]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(train_data_v1, m)

In [ ]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(dev_data_v1, m)

### Train and run simple DistilBERT model

In [8]:
importlib.reload(model_utils_train)
importlib.reload(train_utils)
m = model_utils_train.SimpleDistilBERT()
result = train_utils.train(train_data=train_data_v1_debug, val_data=train_data_v1_debug, model=m, 
                           patience=10, lr=3e-5, print_freq=10)

Epoch: 0, LR: 3e-05, Train Loss: 10.5282, Val Loss: 9.8157, Val f1 7.931, epoch time: 1.7s
Epoch: 10, LR: 3e-05, Train Loss: 4.3554, Val Loss: 4.1090, Val f1 21.738, epoch time: 1.5s
Epoch: 20, LR: 3e-05, Train Loss: 3.0646, Val Loss: 3.2321, Val f1 40.183, epoch time: 1.5s
Epoch: 30, LR: 3e-05, Train Loss: 1.2089, Val Loss: 0.8216, Val f1 83.022, epoch time: 1.6s
Epoch: 40, LR: 3e-05, Train Loss: 0.4165, Val Loss: 0.2666, Val f1 86.909, epoch time: 1.5s
Epoch: 50, LR: 3e-05, Train Loss: 0.4213, Val Loss: 0.1705, Val f1 88.320, epoch time: 1.5s


In [ ]:
importlib.reload(model_utils_train)
importlib.reload(train_utils)
m = model_utils_train.SimpleDistilBERT()
result = train_utils.train(train_data=train_data_v1, val_data=dev_data_v1, model=m, 
                           patience=10, lr=3e-5, print_freq=1, print_batch=True)

batch 0,estimated epoch time: 1712.0642431259155s
batch 100,estimated epoch time: 1116.6682387865505s
batch 200,estimated epoch time: 1012.547588968277s
batch 300,estimated epoch time: 1998.0477038383485s
batch 400,estimated epoch time: 914.0839289937701s
batch 500,estimated epoch time: 2028.6648106575012s
batch 600,estimated epoch time: 1615.4702914782933s
batch 700,estimated epoch time: 1689.8450864950817s
batch 800,estimated epoch time: 1200.424531698227s
batch 900,estimated epoch time: 1323.8708548545837s
batch 1000,estimated epoch time: 1807.5487908124924s
batch 1100,estimated epoch time: 1917.021449497768s
batch 1200,estimated epoch time: 1891.019189119339s
batch 1300,estimated epoch time: 1117.0531740188599s
batch 1400,estimated epoch time: 1415.8014078140259s
batch 1500,estimated epoch time: 3068.5418788790703s
batch 1600,estimated epoch time: 1206.836109161377s
batch 1700,estimated epoch time: 1762.0265908241272s
batch 1800,estimated epoch time: 2878.968561331431s
batch 1900,e

batch 15500,estimated epoch time: 1427.4027141571044s
batch 15600,estimated epoch time: 1425.4083486795425s
batch 15700,estimated epoch time: 1882.2890184402465s
batch 15800,estimated epoch time: 2138.485228538513s
batch 15900,estimated epoch time: 2176.3778251012163s
batch 16000,estimated epoch time: 1763.2234186172484s
batch 16100,estimated epoch time: 1432.2734279632568s
batch 16200,estimated epoch time: 1702.2102219581604s
batch 16300,estimated epoch time: 2390.1422502994537s
batch 16400,estimated epoch time: 1379.3794776916504s
batch 16500,estimated epoch time: 1983.452242946625s
batch 16600,estimated epoch time: 1529.53340921402s
batch 16700,estimated epoch time: 1645.5808762311935s
batch 16800,estimated epoch time: 1633.5370146632195s
batch 16900,estimated epoch time: 1023.8414639366997s
batch 17000,estimated epoch time: 1828.804299513499s
batch 17100,estimated epoch time: 2710.7565695762632s
batch 17200,estimated epoch time: 2025.646677851677s
batch 17300,estimated epoch time: 

In [3]:
train_data_v1.num_batch

18865